In [18]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
import h5py
from tensorflow.keras import backend as K

sns.set()
tf.enable_eager_execution()
tf.set_random_seed(1867)

In [20]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'),
    tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'),
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'),
    tf.keras.layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [22]:
def test(model, dataset):
    epoch_accuracy = tf.contrib.eager.metrics.Accuracy()
    for x, y in dataset:
        outputs = model(x)
        epoch_accuracy(tf.argmax(outputs, axis=1, output_type=tf.int64), y)
    return epoch_accuracy.result().numpy()

In [29]:
optimizer = tf.train.AdamOptimizer()
global_step = tf.train.get_or_create_global_step()

training_losses = []
training_accuracies = []

epochs = 10


model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=256,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
"""
for e in range(epochs):
    epoch_loss_avg = tf.contrib.eager.metrics.Mean()
    epoch_accuracy = tf.contrib.eager.metrics.Accuracy()
    for x, y in dataset_train:
        with tf.GradientTape() as tape:
            outputs = model(x)
            loss = tf.losses.softmax_cross_entropy(tf.one_hot(y, 10), outputs)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights), global_step)
        epoch_loss_avg(loss)
        epoch_accuracy(tf.argmax(outputs, axis=1, output_type=tf.int64), y)
    training_losses.append(epoch_loss_avg.result())
    training_accuracies.append(epoch_accuracy.result())
    
model.save("./LeNet5_results/before_pruning.h5")

test(model, dataset_test)
"""

Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
60000/60000 [==============================] - 36s 599us/sample - loss: 0.2704 - acc: 0.9200 - val_loss: 0.0738 - val_acc: 0.9777
Epoch 2/10
60000/60000 [==============================] - 38s 639us/sample - loss: 0.0638 - acc: 0.9800 - val_loss: 0.0523 - val_acc: 0.9819
Epoch 3/10
60000/60000 [==============================] - 42s 692us/sample - loss: 0.0413 - acc: 0.9872 - val_loss: 0.0441 - val_acc: 0.9860
Epoch 4/10
60000/60000 [==============================] - 41s 675us/sample - loss: 0.0319 - acc: 0.9897 - val_loss: 0.0431 - val_acc: 0.9866
Epoch 5/10
60000/60000 [==============================] - 40s 669us/sample - loss: 0.0232 - acc: 0.9927 - val_loss: 0.0507 - val_acc: 0.9850
Epoch 6/10
60000/60000 [==============================] - 41s 681us/sample - loss: 0.0206 - acc: 0.9932 - val_loss: 0.0381 - val_acc: 0.9892
Epoch 7/10
60000/60000 [==============================] 

'\nfor e in range(epochs):\n    epoch_loss_avg = tf.contrib.eager.metrics.Mean()\n    epoch_accuracy = tf.contrib.eager.metrics.Accuracy()\n    for x, y in dataset_train:\n        with tf.GradientTape() as tape:\n            outputs = model(x)\n            loss = tf.losses.softmax_cross_entropy(tf.one_hot(y, 10), outputs)\n        grads = tape.gradient(loss, model.trainable_weights)\n        optimizer.apply_gradients(zip(grads, model.trainable_weights), global_step)\n        epoch_loss_avg(loss)\n        epoch_accuracy(tf.argmax(outputs, axis=1, output_type=tf.int64), y)\n    training_losses.append(epoch_loss_avg.result())\n    training_accuracies.append(epoch_accuracy.result())\n    \nmodel.save("./LeNet5_results/before_pruning.h5")\n\ntest(model, dataset_test)\n'

In [30]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.030759480433717182
Test accuracy: 0.9917


In [ ]:
# Plot training curves
plt.figure(1, figsize=(15,3))
plt.subplot(121)
plt.plot(training_losses)
plt.xlabel('Epoch')
plt.ylabel('Train Loss')
plt.subplot(122)
plt.plot(training_accuracies)
plt.xlabel('Epoch')
plt.ylabel('Train Accuracy');

In [105]:
def prune_weights(dense_model, percentile):
    prev_kept_columns = None
    pruned_model = tf.keras.models.Sequential()
    pruned_model.add(tf.keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(28,28,1), padding="same"))
    pruned_model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
    pruned_model.add(tf.keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    pruned_model.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
    pruned_model.add(tf.keras.layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding='valid'))
    pruned_model.add(tf.keras.layers.Flatten())
    pruned_model.add(tf.keras.layers.Dense(84, activation='relu'))

    num_layers = len(dense_model.trainable_weights)

    print (num_layers)
    for i_layer, weights in enumerate(dense_model.trainable_weights):
        print (i_layer)
        weights_np = weights.numpy()
        
        # Remove pruned columns
        if i_layer < num_layers-1: # Do not prune last layer
            column_norms = np.linalg.norm(weights_np, ord=2, axis=0)
            critical_value = np.percentile(column_norms, percentile)
            keep_mask = column_norms >= critical_value
            weights_np = weights_np[:, keep_mask]
            
        # Remove rows corresponding to previous layer's pruned columns
        if prev_kept_columns is not None:
            weights_np = weights_np[prev_kept_columns, :]
        
        # Record which columns were kept
        if i_layer < num_layers-1: # No pruned columns in last layer
            prev_kept_columns = np.argwhere(keep_mask).reshape(-1)
        
        # Add new layer to sparse model
        if (i_layer == num_layers - 2):
            new_layer = tf.keras.layers.Dense(weights_np[0].shape, activation='relu')
            pruned_model.add(new_layer)
            new_layer.set_weights([weights_np])
    
    pruned_model.add(tf.keras.layers.Dense(10, activation='softmax')) 
    return pruned_model

In [106]:
pruned_model = prune_weights(model, 50)

#pruned_model.save("./LeNet300_100_results/after_pruning.h5")
score = test(pruned_model, dataset_test)
print('Test accuracy:', score)

10
0
1
2
3
4
5
6
7
8


TypeError: 'tuple' object is not callable

In [ ]:
for e in range(epochs):
    epoch_loss_avg = tf.contrib.eager.metrics.Mean()
    epoch_accuracy = tf.contrib.eager.metrics.Accuracy()
    for x, y in dataset_train:
        with tf.GradientTape() as tape:
            outputs = pruned_model(x)
            loss = tf.losses.softmax_cross_entropy(tf.one_hot(y, 10), outputs)
        grads = tape.gradient(loss, pruned_model.trainable_weights)
        optimizer.apply_gradients(zip(grads, pruned_model.trainable_weights), global_step)
        epoch_loss_avg(loss)
        epoch_accuracy(tf.argmax(outputs, axis=1, output_type=tf.int64), y)
    training_losses.append(epoch_loss_avg.result())
    training_accuracies.append(epoch_accuracy.result())
    
pruned_model.save("./LeNet300_100_results/pruned_retrained.h5")
test(pruned_model,dataset_test)

In [ ]:
percentiles = [10, 20, 30, 40, 50]

pruned_unit_test_losses = []
pruned_unit_test_accuracies = []
pruned_unit_timings = []

for percentile in percentiles:
    sparse_model = prune_weights(model, percentile)
    t = time.clock()
    l, a = test(sparse_model, dataset_test)
    t = time.clock() - t
    print(f"Pruning p{percentile} -- Test Loss: {l:.4f}, Test Accuracy: {a:.4f}, Timing: {t:.2f}s")
    pruned_unit_test_losses.append(l)
    pruned_unit_test_accuracies.append(a)
    pruned_unit_timings.append(t)

In [ ]:
plt.figure(4, figsize=(15,3))
plt.subplot(121)
plt.plot(percentiles, pruned_unit_test_accuracies)
plt.xlabel('Sparsity (%)')
plt.ylabel('Test Accuracy')
plt.subplot(122)
plt.plot(percentiles, pruned_unit_test_losses)
plt.xlabel('Sparsity (%)')
plt.ylabel('Test Loss');